# Preparing the launch file

In [1]:
import numpy as np
import pandas as pd


In [108]:
#pd.read_csv("./source-db/mp-160/POSCAR-unitcell")

pos_path = './source-db/mp-160/POSCAR-unitcell'
sample_path = './launch_thermal_si.py'
pos_file = open(pos_path,"r")

scaled_coordinates=open('./source-db/mp-160/scaled_coordinates.dat', "w")
linenumber=0
for line in pos_file.readlines():
    linenumber=linenumber+1
    if 3<=linenumber<=5:
        print("[",line,"],")


    if linenumber>=9:
        scaled_coordinates.write("("+line+"),\n")
    

scaled_coordinates.close()

[      4.8786175399999996    0.0000000000000000    0.0000000000000000
 ],
[     -2.4393087699999998    4.2250067299999996    0.0000000000000000
 ],
[      0.0000000000000000    0.0000000000000000   12.5104310900000009
 ],
